In [1]:
import pandas as pd
import requests
from io import StringIO
import random
import time
import re
import numpy as np
import urllib.parse as urllib
import urllib.request as urllib2
import json
import time
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans

In [3]:
df_job = pd.read_csv('C:\\Users\\Double 8\\A7_data\\guishan_job.csv')
df_clinic = pd.read_csv('C:\\Users\\Double 8\\A7_data\\guishan_clinic.csv')
df_restaurant = pd.read_csv('C:\\Users\\Double 8\\A7_data\\guishan_restaurant.csv')
df_viewpoint = pd.read_csv('C:\\Users\\Double 8\\A7_data\\guishan_viewpoint.csv')

In [4]:
df_job.columns

Index(['job', 'company', 'address', 'lat', 'lng', 'distance'], dtype='object')

In [5]:
df_clinic.columns

Index(['clinic_address', 'lat', 'lng'], dtype='object')

In [6]:
df_restaurant.columns

Index(['restaurant', 'restaurant_address', 'lat', 'lng', 'distance'], dtype='object')

In [7]:
df_viewpoint.columns

Index(['viewpoint_address', 'lat', 'lng'], dtype='object')

In [8]:
from math import radians, cos, sin, asin, sqrt
 
def haversine(df): # 经度1，纬度1，经度2，纬度2 （十进制度数）
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    s = []
    for i in range(len(df)):
        a7_lng = 121.3852857
        a7_lat = 25.0442435
        # 将十进制度数转化为弧度
        lon1, lat1, lon2, lat2 = map(radians, [df.iloc[i]['lng'], df.iloc[i]['lat'], a7_lng, a7_lat])
        # haversine公式
        dlon = lon2 - lon1 
        dlat = lat2 - lat1 
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a)) 
        r = 6371 # 地球平均半径，单位为公里
        s.append(c * r)
    df['distance'] = s
    return df

In [9]:
df_job

,job,company,address,lat,lng,distance
0,桃園廠研磨中心作業員B (在桃園工作，無須出差，做四休二班制需配合輪班),尖點科技股份有限公司,桃園市龜山區山鶯路169號,24.985186,121.329531,8.642162
1,國外業務人員,拓誠機械股份有限公司,桃園市龜山區德明路同昌巷4號(近銘傳大學),24.983286,121.343865,7.960088
2,營業專員/營業工程師,華儲物流設備有限公司,桃園市龜山區工四工業區頂湖二街62號,25.040319,121.360924,2.492744
3,塗佈機技術員,佑星興業有限公司,桃園市龜山區振興路1173之1號,25.030547,121.376200,1.776864
4,倉儲物流主管,順立智慧股份有限公司,桃園市龜山區忠義路三段159-4號(C倉),25.062569,121.354357,3.722722
5,桃園廠研磨中心作業員A (歡迎有意赴日本東京發展的求職者，做四休二班制需配合輪班),尖點科技股份有限公司,桃園市龜山區山鶯路169號,24.985186,121.329531,8.642162
6,國外業務營業代表(飛宏科技),飛宏科技股份有限公司,桃園市龜山區復興三路568號(林口華亞科技園區),25.039684,121.372178,1.414502
7,電商美編/平面設計人員,翔耀實業股份有限公司,桃園市龜山區東舊路街118巷11號,25.012839,121.368641,3.873841
8,繪圖設計工程師,宇盟實業有限公司,桃園市龜山區頂湖路55號,25.037937,121.365088,2.152259
9,內勤行銷業務(龜山/林口),沃肯科技有限公司,桃園市龜山區,25.019909,121.365599,3.354955


In [11]:
2895+4568+1724

9187